# Lineare Regression

In diesem Notebook werden mittels linearer Regression Vorhersagen auf dem "Advertising"-Datensatz machen. Ziel ist es auf Basis von Werbeausgaben (im Bereich "TV", "Radio" und "Newspaper") Vorhersagen über Verkaufserlöse ("Sales") zu machen.

### Laden des Advertising-Datensatzes

Zuerst laden wird die Daten aus der csv-Datei `advertising.csv` in einen Pandas-DataFrame und schauen uns die Daten kurz an.

In [1]:
import pandas as pd
data_raw = pd.read_csv("data/advertising.csv")
data_raw.head()

FileNotFoundError: [Errno 2] No such file or directory: 'data/advertising.csv'

Die `head`-Funktion zeigt nur die ersten 5 Datenpunkte im DataFrame an. Um zu wissen wie viele Datenpunkte sich im DataFrame befinden, schauen wir auf das `shape`-Attribut.

In [ ]:
rows, cols = data_raw.shape
print("Anzahl Zeilen:", rows)
print("Anzahl Spalten:", cols)

Die erste Spalte enthält lediglich einen fortlaufenden Index und wird für die Vorhersage nicht benötigt, daher wird sie entfernt.

In [ ]:
data = data_raw.drop(columns=['index'])
data.head()

Als nächstes visualieren wir die Datenpunkte mit Hilfe der `matplotlib`-Library.
Dazu erstellten wir einen Plot, welcher auf der x-Achse die `TV`-Daten und auf der y-Achse die `sales`-Daten darstellt.

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(16, 8))
plt.scatter(data['TV'], data['sales'])
plt.xlabel("TV Werbebudget (€)")
plt.ylabel("Sales (€)")
plt.show()

### Training der linearen Regression

Als erstes Modell trainieren wir eine lineare Regression mit nur einem Feature. Als Feature wählen wir die Spalte `TV`.

Bevor wir mit dem Training beginnen, unterteilten wir die verfügbaren Daten in Trainings- und Testdaten, wobei die Trainingsdaten 80% der ursprünglichen Daten beinhalten sollen und die Testdaten 20%.

In [ ]:
train_data = data.sample(frac=0.8, random_state=0)
test_data = data.drop(train_data.index) # Daten welche nicht in train_data sind

print('Shape der Trainingsdaten:', train_data.shape)
print('Shape der Testdaten:', test_data.shape)

Anschließend trainieren wir auf den Trainingsdaten eine lineare Regression mit dem Feature `TV` und dem Label `sales`.
Dafür erstellen wir:
1. Einen DataFrame mit dem Feature `TV`. Diesen nennen wir `X_train`
2. Eine Series mit dem Label. Diese nennen wir `y_train`

Um `X_train` als DataFrame und nicht als Series zu erhalten, müssen wir `TV` als Teil einer Liste übergeben. Der folgende Code zeigt den Unterschied:

In [ ]:
X_series = train_data['TV'] # nur TV selektiert
print("Datentyp von X_series:", type(X_series))
X_df = train_data[['TV']] # Liste mit TV als einzigem Element
print("Datentyp von X_df:", type(X_df))

X_train = X_df # Die Features müssen als DataFrame vorliegen und nicht als Series
y_train = train_data['sales']
print("Datentyp von y_train:", type(y_train))

Jetzt folgt das eigentliche Training des Modells:

In [ ]:
from sklearn.linear_model import LinearRegression

reg = LinearRegression()
reg.fit(X_train, y_train)

Die lineare Regression ist nun trainiert und die Modellgewichte in the `reg`-Variable verfügbar. Wir können uns nun die Regressionsgerade ausgeben lassen.

In [ ]:
print(f"Regressionsgerade: y = {reg.intercept_} + {reg.coef_[0]}*TV")

Mit dem trainierten Modell können wir nun Vorhersagen auf einzelnen Datenpunkten machen.

In [ ]:
dataPoint = X_train.iloc[0] # erster Datenpunkt aus den Trainingsdaten
prediction = reg.predict([dataPoint]) # predict-Methode erwartet Liste von Datenpunkten 
print(f"Bei einem TV-Werbebudget von {dataPoint[0]}€, werden {prediction[0]}€ Umsatz erzielt.")

Um zu Visualisieren wie die trainierte Regressionsgerade aussieht, machen wir mit dem Modell Vorhersagen auf den Trainingsdatenpunkten.

In [ ]:
prediction_train = reg.predict(X_train) # Vorhersage auf allen Trainingsdaten gleichzeitig

plt.figure(figsize=(16, 8))
plt.scatter(data['TV'], data['sales']) # Trainingsdatenpunkte
plt.plot(X_train, prediction_train, 'r') # Regressionsgerade
plt.xlabel("TV Werbebudget ($)")
plt.ylabel("Umsatz (Euro)")
plt.show()

### Testen des Regressionsmodells

Um die Qualität des trainierte Regressionsmodells zu überprüfen, machen wir damit Vorhersagen auf den Testdaten und bestimmen den MSE.

In [ ]:
from sklearn.metrics import mean_squared_error
X_test = test_data[['TV']]     # X_test muss ein DateFrame sein
y_test = test_data['sales']    # y_test muss eine Series sein
prediction_test = reg.predict(X_test)
mse_test =  mean_squared_error(y_test, prediction_test)
print("Mean squared error (MSE) auf Testdaten:", mse_test)

### Multidimensionale lineare Regression

Wir erweitern nun die lineare Regression indem wir die beiden Features `radio` und `newspaper` zusätzlich benutzen.

In [ ]:
X_train = train_data[["TV", "radio", "newspaper"]]
y_train = train_data['sales']
reg_all = LinearRegression()
reg_all.fit(X_train, y_train)
print(f"Regression: Y = {reg_all.intercept_} + {reg_all.coef_[0]}*TV + {reg_all.coef_[1]}*radio + {reg_all.coef_[2]}*newspaper")

Abschließend nutzen wir das neuen Modell um wiederum Vorhersagen auf den Testdaten zu machen.

In [ ]:
X_test = test_data[["TV", "radio", "newspaper"]]
y_test = test_data['sales']
predictions = reg_all.predict(X_test)
mse =  mean_squared_error(y_test, predictions)
print("Mean squared error (MSE) auf Testdaten: %.2f" % mse)

Wie wir sehen können ist die Vorhersage für das multidimensionale Modell besser als die normale lineare Regression (der MSE ist wesentlich kleiner). Das Hinzufügen der neuen Features verbessert also die Vorhersagekraft des Modells. 